<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_MultiTimeFrame_RSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2024-05-01 16:50:06


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta

In [3]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd

#mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')
  return mypps_df

mypps_df =  get_mypps_df()
mypfs_df =  get_mypfs_df()

def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ', 'EQUITAS']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

def get_stock_rsi(symbols, period='5y'):
    result = []
    for symbol in symbols:
        # Fetch data from Yahoo Finance
        stock_data = yf.download(symbol + ".NS", period=period)

        close_price = round(stock_data.tail(1)['Close'].values[0],2)
        #print(close_price)
        all_time_high = round(max(stock_data['Close']),2)
        #print(all_time_high)
        ath_perc = round((((all_time_high - close_price)*100)/close_price),0)
        #print(ath_perc)

        # Calculate daily RSI
        stock_data.ta.rsi(close='Close', length=14, append=True)

        # Calculate weekly RSI
        weekly_data = stock_data.resample('W').last()
        weekly_data.ta.rsi(close='Close', length=14, append=True)

        # Calculate monthly RSI
        monthly_data = stock_data.resample('M').last()
        monthly_data.ta.rsi(close='Close', length=14, append=True)

        # Extract the last value of RSI for each timeframe
        daily_rsi = round(stock_data['RSI_14'].iloc[-1], 0)
        weekly_rsi = round(weekly_data['RSI_14'].iloc[-1], 0)
        monthly_rsi = round(monthly_data['RSI_14'].iloc[-1], 0)

        # Append the results to the list
        result.append({
            'Stock_Symbol': symbol,
            'Daily_RSI': daily_rsi,
            'Weekly_RSI': weekly_rsi,
            'Monthly_RSI': monthly_rsi,
            'ATH%' : ath_perc
        })

    # Create a DataFrame with the results
    result_df = pd.DataFrame(result)
    return result_df

if __name__ == "__main__":
    # Example usage
    mypps_lst = get_stock_ids(mypps_df)
    symbols = mypps_lst # Example NSE stock symbols
    stock_rsi = get_stock_rsi(symbols)
stock_rsi

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Stock_Symbol,Daily_RSI,Weekly_RSI,Monthly_RSI,ATH%
0,3MINDIA,54.0,48.0,56.0,22.0
1,5PAISA,47.0,51.0,63.0,38.0
2,AARTIIND,60.0,66.0,60.0,56.0
3,AAVAS,66.0,58.0,47.0,97.0
4,ABBOTINDIA,49.0,54.0,64.0,12.0
...,...,...,...,...,...
225,WHIRLPOOL,65.0,62.0,48.0,78.0
226,WIPRO,41.0,49.0,54.0,56.0
227,WSTCSTPAPR,67.0,53.0,58.0,17.0
228,YESBANK,59.0,59.0,62.0,573.0


In [4]:
tmp_df = stock_rsi[(stock_rsi['Daily_RSI'] > 60) & (stock_rsi['Daily_RSI'] < 70)]
tmp_df = tmp_df[(stock_rsi['Weekly_RSI'] > 55) & (tmp_df['Weekly_RSI'] < 65)]
tmp_df = tmp_df[(stock_rsi['Monthly_RSI'] > 50) & (tmp_df['Monthly_RSI'] < 60)]
tmp_df['InPortfolio'] = tmp_df['Stock_Symbol'].isin(mypfs_df['Symbol']).astype(int)
tmp_df.sort_values(by = 'ATH%', ascending=False)

<ipython-input-4-40b36d755ed8>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp_df = tmp_df[(stock_rsi['Weekly_RSI'] > 55) & (tmp_df['Weekly_RSI'] < 65)]
<ipython-input-4-40b36d755ed8>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp_df = tmp_df[(stock_rsi['Monthly_RSI'] > 50) & (tmp_df['Monthly_RSI'] < 60)]


,Stock_Symbol,Daily_RSI,Weekly_RSI,Monthly_RSI,ATH%,InPortfolio
103,IEX,63.0,58.0,54.0,89.0,1
37,BIOCON,64.0,60.0,54.0,62.0,1
204,TEAMLEASE,62.0,62.0,57.0,59.0,1
57,CUB,69.0,63.0,55.0,52.0,0
104,IGL,62.0,59.0,56.0,26.0,0
60,DEEPAKNTR,66.0,60.0,59.0,20.0,0
